<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/GPU_QM_DM_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install --upgrade  git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

  Cloning https://github.com/fagonzalezo/qmc.git to /tmp/pip-req-build-oh3159r7
  Running command git clone -q https://github.com/fagonzalezo/qmc.git /tmp/pip-req-build-oh3159r7
  Created wheel for qmc: filename=qmc-0.0.1-cp37-none-any.whl size=11811 sha256=37fb70fd8efb1f5f93a6c56c4b514e62b97dd04ba358e104e9c3d4ada094e844
  Stored in directory: /tmp/pip-ephem-wheel-cache-b71i5z5e/wheels/ff/c2/53/58cb50bb85949fb1dcda723a7c1f48a590515dee0aa40a4cc2
Successfully built qmc


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import qmc.tf.layers as layers
import qmc.tf.models as models

In [3]:
from keras.datasets import mnist
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((60000,784))
X_test = X_test.reshape((10000,784))

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

transformer = OneHotEncoder(sparse=False)
y_train_bin = transformer.fit_transform(y_train[:, np.newaxis])

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
shape X_train :  (60000, 784)
shape y_train :  (60000,)
shape X_test :  (10000, 784)
shape y_test :  (10000,)


In [4]:
BATCH_SIZE = 256

model = tf.keras.models.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(128,activation='relu'),
tf.keras.layers.Dense(10),
tf.keras.layers.Softmax()
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)
EPOCHS = 10
  
history = model.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 4s 6ms/step - loss: 0.8492 - categorical_accuracy: 0.7724 - val_loss: 0.2571 - val_categorical_accuracy: 0.9277
Epoch 2/10
188/188 [==============================] - 1s 3ms/step - loss: 0.2401 - categorical_accuracy: 0.9329 - val_loss: 0.1862 - val_categorical_accuracy: 0.9502
Epoch 3/10
188/188 [==============================] - 1s 3ms/step - loss: 0.1768 - categorical_accuracy: 0.9501 - val_loss: 0.1597 - val_categorical_accuracy: 0.9538
Epoch 4/10
188/188 [==============================] - 1s 3ms/step - loss: 0.1409 - categorical_accuracy: 0.9591 - val_loss: 0.1381 - val_categorical_accuracy: 0.9599
Epoch 5/10
188/188 [==============================] - 1s 3ms/step - loss: 0.1168 - categorical_accuracy: 0.9675 - val_loss: 0.1219 - val_categorical_accuracy: 0.9657
Epoch 6/10
188/188 [==============================] - 1s 3ms/step - loss: 0.1018 - categorical_accuracy: 0.9708 - val_loss: 0.1122 - val_categorical_accuracy: 0.9682
Epoc

In [5]:
BATCH_SIZE = 256
input_dim = 784
component_dim = 128
gamma = 2**-5
num_eig = 128
random_state = 0

inputs = tf.keras.Input(shape=(784,))
fm_x = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc = layers.QMeasureDMClassifEig(dim_x=component_dim , dim_y=10, eig_out=num_eig, num_eig=num_eig)
rho_y = qmdmc(rho_x)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
qmdmclf.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

EPOCHS = 10
  
history = qmdmclf.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 10s 44ms/step - loss: 1.9797 - accuracy: 0.3813 - val_loss: 1.2974 - val_accuracy: 0.7262
Epoch 2/10
188/188 [==============================] - 8s 43ms/step - loss: 1.2429 - accuracy: 0.7439 - val_loss: 1.1097 - val_accuracy: 0.7938
Epoch 3/10
188/188 [==============================] - 8s 43ms/step - loss: 1.0917 - accuracy: 0.7941 - val_loss: 1.0360 - val_accuracy: 0.7533
Epoch 4/10
188/188 [==============================] - 8s 43ms/step - loss: 1.0412 - accuracy: 0.7414 - val_loss: 1.0055 - val_accuracy: 0.7355
Epoch 5/10
188/188 [==============================] - 8s 43ms/step - loss: 1.0095 - accuracy: 0.7322 - val_loss: 0.9952 - val_accuracy: 0.7232
Epoch 6/10
188/188 [==============================] - 8s 43ms/step - loss: 0.9977 - accuracy: 0.7278 - val_loss: 0.9866 - val_accuracy: 0.7174
Epoch 7/10
188/188 [==============================] - 8s 43ms/step - loss: 0.9831 - accuracy: 0.7287 - val_loss: 0.9781 - val_accuracy: 0.750

In [6]:
BATCH_SIZE = 128
input_dim = 784
component_dim = 128
gamma = 2**-5
num_eig = 128
random_state = 0
inputs = tf.keras.Input(shape=(784,))
fm_x1 = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x1(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc1 = layers.QMeasureDMClassifEig(dim_x=component_dim , dim_y=10, eig_out=num_eig, num_eig=num_eig)
rho_h = qmdmc1(rho_x)
qmdmc2 = layers.QMeasureDMClassifEig(dim_x=10 , dim_y=10, eig_out=20, num_eig=10)
rho_y = qmdmc2(rho_h)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf2 = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6)
qmdmclf2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

fm_x1.set_weights(fm_x.get_weights())
fm_x1.trainable = False
qmdmc1.set_rho(qmdmc.get_rho())
qmdmc1.trainable = False
EPOCHS = 10
  
history = qmdmclf2.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 11s 24ms/step - loss: 2.2522 - accuracy: 0.2541 - val_loss: 1.7966 - val_accuracy: 0.4552
Epoch 2/10
375/375 [==============================] - 8s 22ms/step - loss: 1.7367 - accuracy: 0.5218 - val_loss: 1.6306 - val_accuracy: 0.6287
Epoch 3/10
375/375 [==============================] - 8s 22ms/step - loss: 1.5913 - accuracy: 0.6378 - val_loss: 1.5654 - val_accuracy: 0.5886
Epoch 4/10
375/375 [==============================] - 9s 23ms/step - loss: 1.5504 - accuracy: 0.5868 - val_loss: 1.5466 - val_accuracy: 0.5419
Epoch 5/10
375/375 [==============================] - 9s 23ms/step - loss: 1.5290 - accuracy: 0.5100 - val_loss: 1.5286 - val_accuracy: 0.4252
Epoch 6/10
375/375 [==============================] - 9s 23ms/step - loss: 1.5296 - accuracy: 0.4173 - val_loss: 1.5346 - val_accuracy: 0.4348
Epoch 7/10
375/375 [==============================] - 9s 23ms/step - loss: 1.5270 - accuracy: 0.4288 - val_loss: 1.5057 - val_accuracy: 0.445

In [ ]:
qmdmclf2.summary()

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 784)]        0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_55 (Sl (None, 1)            0           input_20[0][0]                   
__________________________________________________________________________________________________
tf.ones_like_19 (TFOpLambda)    (None, 1)            0           tf.__operators__.getitem_55[0][0]
__________________________________________________________________________________________________
q_feature_map_rff_19 (QFeatureM (None, 128)          100480      input_20[0][0]                   
___________________________________________________________________________________________